# Modélisation
### Dans ce notebook nous allons suivre exactement la même stratégie que le notebook 2_modelisation en utililsant les données sans réduction par PCA

In [1]:
import pandas as pd
# ^^^ pyforest auto-imports - don't write above this line
import pyforest
import os
import warnings
import sklearn
import time 
import pickle

import matplotlib.pyplot as plt
import seaborn as sns


#preparation des données
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

#modeles :
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import RidgeClassifier, LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier

#mesure performance : 
from sklearn import metrics
warnings.filterwarnings("ignore")

In [2]:
# Lecture des données :
path1 = [i for i in os.getcwd().split("\\")]
path1

['D:',
 'Etude_Data_science',
 'Kaggle_competition',
 '02_Binary_Prediction_of_Smoker_Status_using_Bio-Signals',
 'notebook']

In [3]:
path2 = ("\\").join(path1[:-1]) + "\\dataset"

In [4]:
os.listdir(path2)

['data_cleaned.csv',
 'data_cleaned_PCA.csv',
 'data_cleaned_withoutPCA.csv',
 'playground-series-s3e24.zip',
 'sample_submission.csv',
 'test.csv',
 'train.csv']

In [5]:
data = pd.read_csv(path2+"\\data_cleaned_withoutPCA.csv", index_col = 0)
df = data.copy()
df.head()

<IPython.core.display.Javascript object>

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),systolic,relaxation,fasting blood sugar,Cholesterol,...,hemoglobin,serum creatinine,AST,ALT,Gtp,hearing(left),hearing(right),Urine protein,dental caries,smoking
60700,-0.363764,-1.729695,-1.361653,-0.333247,-1.261648,-1.278404,-0.194342,0.127119,0.239934,-0.591152,...,-2.090771,-1.626365,-1.190789,-0.851755,-0.871202,1,1,1,0,0
44065,1.750195,-1.729695,-1.361653,-1.563035,-0.762843,-0.002432,1.298727,1.348476,9.462864,0.500179,...,1.052672,-1.626365,-0.880976,-0.139049,-0.358082,1,1,1,0,0
39538,0.904612,-1.162983,-0.963707,-0.333247,0.484171,0.507957,-1.058751,-0.650109,-0.610407,-1.471258,...,-0.973102,-1.626365,-0.674434,-0.632461,-0.646712,1,1,1,0,0
105427,0.904612,-0.596270,-0.565762,0.002149,-0.762843,-0.512820,0.591484,0.349184,-0.610407,1.204263,...,-0.134851,-1.071018,-1.190789,-0.742108,-0.326012,1,1,1,0,0
148669,-1.209348,1.670580,1.821910,1.343736,-0.014635,0.507957,-0.194342,0.571249,-0.741229,-1.154420,...,-0.693685,0.039675,-0.054807,0.189892,-0.486362,1,1,1,0,0


#### Récupération d'un sample du dataframe pour effectuer les différents test de modélisation

##### La colonne sera ensuite mute une fois les tests terminés pour utiliser l'ensemble du dataframe
df = df.sample(10000, random_state=42)

In [7]:
df.shape

(159256, 23)

In [8]:
X,y = df.drop("smoking", axis = 1), df["smoking"]
X_train,X_test,y_train, y_test = train_test_split(X,y, random_state=42,
                                                 test_size=0.2)

# Plan : 

## 1. Création de fonction de modélisation et  Préparation des modèles

## 2. Entrainement des modèles

## 3. Résultats et premières selection de modèles

## 4. Visualisation des autres metrics

## 5. Amélioration des modèles par GridSearchCV
## 6. Visualisation des performances

# 1. Création des fonctions de modélisation et préparation des modèles

In [ ]:
all_model = [DummyClassifier(),
             LogisticRegression(),
             KNeighborsClassifier(),
             SGDClassifier(),
             SVC(),
             RandomForestClassifier()]

In [ ]:
### Création d'une fonction d'entrainement pour un modèle donné

def train_model(model):
    """le modèle sera entrainé par validation score sur 5 split
    et affichage du score moyen.
    la fonction renvoi les prédiction du modèle entrainé et le temps d'entrainement"""
    scores = (cross_val_score(estimator=model, X=X_train,y=y_train, cv=5)).mean()
    print(str(model))
    print("Score moyen validation croisée :", scores.round(2))
    
    # Mesurer le temps de début
    start_time = time.time()
    
    #Entrainement du modele
    model.fit(X_train,y_train)
    
    # Mesurer le temps de fin
    end_time = time.time()
    
    # Calculer la durée totale
    training_time = round(end_time - start_time,2)
    #Convertir la duree en minute :
    training_time_min = round(training_time/60,2)

    yp = model.predict(X_test.values) #rajout ici '.values' pour que ca fonctionne
    return yp,training_time_min

In [ ]:
### Création d'une fonction qui renvoi l'ensemble des metrics :

def metric(prediction, model_name, exe_time):
    """Fonction qui renvoi un dataframe contenant les valeurs des metrics
    principales : f1score, accuracy, recall, precision score et le temps d'execution
    à partir de la liste des prédictions d'un modèle"""
    
    f1 = metrics.f1_score(y_true = y_test, y_pred = prediction)
    accuracy = metrics.accuracy_score(y_true = y_test, y_pred = prediction)
    recal = metrics.recall_score(y_true = y_test, y_pred = prediction)
    precision = metrics.precision_score(y_true = y_test, y_pred = prediction)
    
    all_metric = [f1,accuracy,recal,precision, exe_time]
    
    index_metric = ["F1","accuracy","recall","precision","Duree_entrainement"]
    
    series = pd.Series(all_metric, name=str(model_name), index=index_metric)
    return pd.DataFrame(series)
    
    

In [ ]:
### fonction pour créer une matrice de confusion : 

def matrice_confusion(prediction):
    cf = metrics.confusion_matrix(y_true=y_test, y_pred=prediction)
    plt.figure(figsize=(4,4))
    ax = sns.heatmap(cf, annot = True, linewidths=0.8, linecolor="black", fmt = ".0f",cbar=False, cmap = "Blues")
    ax.set_xlabel('Prédictions')
    ax.set_ylabel('Valeurs réelles')
    plt.title("Confusion Matrix")
    plt.show()


In [ ]:
### Fonction pour obtenir la learning curve d'un modèle : 
def learning_curv(model):
    train_sizes, train_scores, test_scores = learning_curve(model,
                                                                X_train, y_train, cv=5)
    
# Calcul des scores moyens pour l'apprentissage et la validation
    train_scores_mean = np.mean(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)

# Tracer les courbes d'apprentissage
    plt.figure(figsize=(4, 3))
    plt.plot(train_sizes, train_scores_mean, label='Score d\'entraînement moyen')
    plt.plot(train_sizes, test_scores_mean, label='Score de validation moyen')
    plt.xlabel('Taille de l\'ensemble d\'entraînement')
    plt.ylabel('Score')
    plt.title('Courbes d\'apprentissage')
    plt.legend()
    plt.show()
    


In [ ]:
### Fonction pour obtenir la courbe ROC : 

def ROC(prediction):
    FP_rate, TP_rate, thresholds = metrics.roc_curve(y_test,prediction)
    roc_auc = metrics.auc(FP_rate,TP_rate)
    print("ROC_AUC =", roc_auc.round(2))
    
    
    ### Création de la courbe AUC ROC :
    plt.figure(figsize=(6,6))
    
    #Courbe prédictive :
    sns.lineplot(x = FP_rate, y = TP_rate, color = "orange", label = f"AUC = {roc_auc:,.2f}%")
    
    #Courbe random :
    sns.lineplot(x=[0,1],y=[0,1], linestyle = "--", c = "r", label= "Random Classifier")

    plt.ylabel("True Positive rate")
    plt.xlabel("False Positive rate")
    plt.title("ROC")
    plt.legend(loc = "upper left")
    plt.show()

In [ ]:
### Fonction permettant d'améliorer un modèle grâce au GridSearchCV :

def Grid(model, param):
    """Le modèle sera entrainé sur 5 splits
    et renverra les prédictions du meilleur modèles ainsi que son temps d'entrainement"""
    
 # Mesurer le temps de début
    start_time = time.time()
        
    #Entrainement du grid sur les paramètres
    grid = GridSearchCV(estimator=model, param_grid=param, cv=5)
    grid.fit(X_train, y_train)
    # Mesurer le temps de fin
    end_time = time.time()
    
    #Recuperation des meilleurs hyper parametres :
    best_model = grid.best_estimator_
    
    #Recuperation des predictions
    yp = best_model.predict(X_test)
    
    #Calcul temps d'execution:
    training_time = round(end_time- start_time  , 3)
    #On converti en minutes :
    training_time_min = round(training_time/60,2)
    
    return yp, training_time_min

## 2. Entrainement des différents modèles et récupération des metrics :
- Nous allons créer une boucle qui va parcourir notre liste des modèles utilisés et qui fera :
    - Entrainement des modèles et récupèrera les prédictions et les temps d'execution
    - Enregistrement du modèle sur pickle
    - Ajout des metrics dans le dictionnaire dict_metric
    - ajout des prédiction dans le dictionnaire dict_prediction
    -  enregistrement final dans un dataframe contenant les différentes metrics

In [9]:
save_path = r'D://Etude_Data_science//Kaggle_competition//02_Binary_Prediction_of_Smoker_Status_using_Bio-Signals/Modeles/\\withoutPCA\\'

In [ ]:
dict_metric = {}
for current_model in all_model:
    prediction, temps_exe = train_model(current_model)
    dict_metric[str(current_model)] = metric(prediction=prediction, model_name=str(current_model), exe_time=temps_exe)
df_metrics = pd.concat(dict_metric.values(), axis = 1)

In [ ]:
#Dictionnaire qui contiendra les différentes metrics de chaque modèle
dict_metric = {}

#dictionnaire contenant les valeurs prédictives pour chaque modèles
dict_prediction = {}

for current_model in all_model:
    #Recuperation des prédictions et du temps d'execution de chaque modèles
    prediction, temps_exe = train_model(current_model)
    
    
    # Récupération du nom du modèle et du chemin d'enregistrement
    model_name = str(current_model)
    model_filename = os.path.join(save_path, f"{model_name}.pkl")
    
    #Enregistrement des modèles préentrainé :
    with open(model_filename, 'wb') as model_file:
        pickle.dump(current_model, model_file)
    
    #Enregistrement des prédiction dans le dictionnaire :
    dict_prediction[model_name] = prediction
    
    # Enregistrement des métriques au dictionnaire
    dict_metric[model_name] = metric(prediction=prediction, model_name=model_name, exe_time=temps_exe)


    
#Pour les Dataframe :
    
# Concaténation des métriques en un DataFrame
df_metrics = pd.concat(dict_metric.values(), axis=1)
#intervertissons les colonnes et les indexs :
df_metrics = df_metrics.T
# Enregistrement du DataFrame des métriques
df_metrics.to_csv("df_metrics.csv")


#Enregistrement des prédictions dans un dataframe :
df_prediction = pd.DataFrame(dict_prediction)
# Enregistrement du DataFrame des prédictions
df_prediction.to_csv("df_prediction.csv")


### 3. Résultats et premières selection de modèles
- Nous allons déja effectuer un premier filtre de modèles en supprimant ceux qui mettent beaucoup trop de temps à s'entrainer

In [ ]:
df_metrics = df_metrics.T
df_metrics.sort_values(by = "F1", ascending=False)

### Observation :
- 

### Conclusion :
- 

In [ ]:
df_prediction = df_prediction.drop(["listemodele_"], axis = 1)

## 4. Visualisation des autres metrics : 

### matrice de confusion : 

In [ ]:
for i in df_prediction:
    print(i)
    matrice_confusion(df_prediction[i])
    

### ROC :

In [ ]:
for i in df_prediction:
    print(i)
    ROC(df_prediction[i])
    

### Observation 
- 

## 5. Amélioration des modèles par GridSearchCV